<a href="https://colab.research.google.com/github/Nataliia5722/Machine-Learning-Spring-2021/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Data preprocessing and logistic regression. Titanic.

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns


In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/Nataliia5722/Machine-Learning-Spring-2021/main/titanic.csv")

In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
data.shape

(891, 12)

In [6]:
data.dropna(axis="columns")

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500
...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,0,0,211536,13.0000
887,888,1,1,"Graham, Miss. Margaret Edith",female,0,0,112053,30.0000
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,1,2,W./C. 6607,23.4500
889,890,1,1,"Behr, Mr. Karl Howell",male,0,0,111369,30.0000


In [7]:
train_data = data.copy()
train_data["Age"].fillna(data["Age"].median(skipna=True), inplace=True)
train_data["Embarked"].fillna(data['Embarked'].value_counts().idxmax(), inplace=True)

In [8]:
training=pd.get_dummies(train_data, columns=["Pclass","Embarked","Sex"])
training.drop('Cabin', axis=1, inplace=True)
training.drop('Sex_female', axis=1, inplace=True)
training.drop('PassengerId', axis=1, inplace=True)
training.drop('Name', axis=1, inplace=True)
training.drop('Ticket', axis=1, inplace=True)

In [9]:
training

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male
0,0,22.0,1,0,7.2500,0,0,1,0,0,1,1
1,1,38.0,1,0,71.2833,1,0,0,1,0,0,0
2,1,26.0,0,0,7.9250,0,0,1,0,0,1,0
3,1,35.0,1,0,53.1000,1,0,0,0,0,1,0
4,0,35.0,0,0,8.0500,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.0,0,0,13.0000,0,1,0,0,0,1,1
887,1,19.0,0,0,30.0000,1,0,0,0,0,1,0
888,0,28.0,1,2,23.4500,0,0,1,0,0,1,0
889,1,26.0,0,0,30.0000,1,0,0,1,0,0,1


In [10]:
x = training.drop('Survived',axis=1)
y = training['Survived']

In [11]:
x.isnull().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Sex_male      0
dtype: int64

In [12]:
x.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male
0,22.0,1,0,7.2500,0,0,1,0,0,1,1
1,38.0,1,0,71.2833,1,0,0,1,0,0,0
2,26.0,0,0,7.9250,0,0,1,0,0,1,0
3,35.0,1,0,53.1000,1,0,0,0,0,1,0
4,35.0,0,0,8.0500,0,0,1,0,0,1,1


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=13)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

lg1 = LogisticRegression(random_state=13, class_weight='balanced')
lg1.fit(X_train,y_train)
y_pred = lg1.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
print(y_pred.sum())

114


###Calculate metrics


In [16]:
#Calculate accuracy using sklearn
acc_log = round(lg1.score(X_train, y_train) * 100, 2)
print('Sklearn accuracy: ', acc_log)

Sklearn accuracy:  79.29


In [17]:
#Confusion Matrix
#from sklearn.metrics import confusion_matrix
#confusion_matrix(y_test, y_pred)
conf = pd.crosstab(y_test.values, y_pred)
conf

col_0,0,1
row_0,,
0,132,39
1,22,75


In [18]:
#Calculate accuracy as using confusion matrix as (TP+TN)/(TP+FP+TN+FN)
accuracy = (conf[0][0]+conf[1][1])/(conf[0][0]+conf[0][1]+conf[1][0]+conf[1][1])
print('Accuracy of the model: ', round(accuracy*100, 2))

Accuracy of the model:  77.24


In [19]:
#Recall sklearn
from sklearn.metrics import recall_score
recall_score(y_test, y_pred)


0.7731958762886598

In [20]:
#Calculate recall as using confusion matrix as TP/(TP+FN)
recall = conf[0][0]/(conf[0][0]+conf[1][0])
recall

0.7719298245614035

In [21]:
#Precision sklearn
from sklearn.metrics import precision_score

precision_score(y_test, y_pred)

0.6578947368421053

In [22]:
#Calculate precision as using confusion matrix as TP/(TP+FP)

precision = conf[0][0]/(conf[0][0]+conf[0][1])
precision

0.8571428571428571

In [23]:
#True negative rate, TNR = TN/(TN+FP)

tnr = conf[1][1]/(conf[1][1]+conf[0][1])
tnr

0.7731958762886598

In [24]:
#F1 score

f1 = 2*(precision*recall)/(precision+recall)
